In [1]:
import pandas as pd
import json
import numpy as np
from os import listdir
from io import StringIO
from Bio import Phylo, AlignIO
from Bio.Alphabet import generic_dna

import re

In [2]:
path="/home/maeperez/scratch/Clams/hyphy2.5/gard_LCB/"
aln_breakdown='../mauve_alignments/core_ALL_alignments/'
path_out="/home/maeperez/projects/def-bacc/maeperez/Clams/VesicSymb_Evolution/dowstream_analyses_and_data/"

genes=[]
for file in listdir(path):
    if file[-4:]=="json":
        genes+=[str(file.split('.')[0])]
genes=sorted(genes)
print(len(genes))


416


In [3]:
path_to_files='../mauve_alignments/core_ALL_alignments/'
with open(path_to_files+'filelist','r') as f:
    lines=[file[8:-6] for file in f.read().splitlines()]

failed_gard=sorted(set(lines)-set(genes))
failed_gard_idx=[lines.index(gene)+1 for gene in failed_gard]
print(len(failed_gard))
print('\n'.join([' '.join(map(str,item)) for item in list(zip(failed_gard_idx,failed_gard))]))
','.join(map(str,failed_gard_idx))


300
11 LCB_011
14 LCB_014
28 LCB_028
40 LCB_040
54 LCB_054
57 LCB_057
74 LCB_074
81 LCB_081
82 LCB_082
86 LCB_086
88 LCB_088
95 LCB_095
96 LCB_096
97 LCB_097
98 LCB_098
99 LCB_099
100 LCB_100
101 LCB_101
103 LCB_103
104 LCB_104
105 LCB_105
106 LCB_106
107 LCB_107
108 LCB_108
110 LCB_110
113 LCB_113
115 LCB_115
122 LCB_122
123 LCB_123
124 LCB_124
125 LCB_125
126 LCB_126
127 LCB_127
128 LCB_128
129 LCB_129
130 LCB_130
132 LCB_132
134 LCB_134
151 LCB_151
155 LCB_155
156 LCB_156
157 LCB_157
158 LCB_158
159 LCB_159
161 LCB_161
162 LCB_162
166 LCB_166
167 LCB_167
168 LCB_168
169 LCB_169
170 LCB_170
171 LCB_171
172 LCB_172
174 LCB_174
175 LCB_175
176 LCB_176
178 LCB_178
179 LCB_179
181 LCB_181
182 LCB_182
183 LCB_183
184 LCB_184
185 LCB_185
186 LCB_186
187 LCB_187
189 LCB_189
190 LCB_190
191 LCB_191
192 LCB_192
194 LCB_194
195 LCB_195
196 LCB_196
197 LCB_197
198 LCB_198
199 LCB_199
200 LCB_200
201 LCB_201
203 LCB_203
206 LCB_206
207 LCB_207
208 LCB_208
210 LCB_210
212 LCB_212
213 LCB_213
214 

'11,14,28,40,54,57,74,81,82,86,88,95,96,97,98,99,100,101,103,104,105,106,107,108,110,113,115,122,123,124,125,126,127,128,129,130,132,134,151,155,156,157,158,159,161,162,166,167,168,169,170,171,172,174,175,176,178,179,181,182,183,184,185,186,187,189,190,191,192,194,195,196,197,198,199,200,201,203,206,207,208,210,212,213,214,216,217,218,220,223,224,225,227,228,229,230,231,232,233,234,235,236,237,238,239,241,249,250,252,253,254,255,256,259,264,265,266,269,270,271,274,276,278,280,284,286,294,295,296,299,300,301,302,307,315,316,317,323,324,325,326,328,329,330,331,332,335,336,337,338,340,341,342,344,346,351,352,354,355,356,367,368,371,376,377,378,379,380,383,384,387,390,391,393,395,396,397,400,402,406,408,409,411,414,417,420,426,427,432,440,441,442,444,447,449,450,451,452,453,454,456,458,463,465,467,468,469,471,472,474,477,479,480,484,486,488,495,498,507,508,511,513,517,519,524,528,529,540,541,543,544,546,548,555,558,563,564,566,569,570,573,574,576,577,578,580,581,582,583,588,592,594,597,598

In [31]:
filenames=[]
with_recombination=[]
for gene in [genes[0]]:
    remote_file = open(path+gene+'.GARD.json')
    gard_json = json.load(remote_file)
    breakpoints=[0]+[gard_json['input']['number of sites']]
    
    if 'improvements' not in gard_json.keys():
        continue
        
    else: 
        best_AICc=sum([gard_json['improvements'][bp]['deltaAICc'] for bp in gard_json['improvements'].keys()])+gard_json['baselineScore']
        AICc_ratio=float(gard_json['singleTreeAICc'])/best_AICc
        if AICc_ratio>=1:
            with_recombination+=[gene]
            
            ### Find all puttive breakpoints
            breakpoints=[0]+[gard_json['breakpointData'][k]['bps'][0][1] for k in gard_json['breakpointData'].keys()]
            print(breakpoints)
            
    ### break alignment at all putative breakpoints ###
    alignment = AlignIO.read(path_to_files+gene+'.fasta', "fasta",alphabet=generic_dna)
    start=breakpoints.pop(0)
    sub=0
    while len(breakpoints)>0:
        sub+=1
        end=breakpoints.pop(0)
        new_aln= alignment[:,start:end]

        for record in new_aln:
            record.id = record.id.split('|')[0]

        new_aln.sort()
        filename=gene+'_'+str(sub)+'_'+str(start)+'to'+str(end)+'.nex'
        filenames+=filename

        AlignIO.write([new_aln],aln_breakdown+'nex/'+filename, "nexus")

        start=end

print( 'number of genes that failed AICc test ',len(with_recombination),"/",len(genes))        

[0, 336, 684, 1390]


NameError: name 'aln_breakdown' is not defined

In [4]:
# with open(path_out+'core_LCB_with_recomb_AICc.txt','w') as f:
#     f.write('\n'.join(with_recombination))